In [2]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [3]:
from fastai.structured import *
from fastai.column_data import *
np.set_printoptions(threshold=50, edgeitems=20)

In [4]:
PATH='data/sales/'

TMP_DIR = f'{PATH}/tmp'
MODEL_DIR = f'{PATH}/model'

os.makedirs(TMP_DIR, exist_ok=True)
os.makedirs(MODEL_DIR, exist_ok=True)
%ls {PATH}

item_categories.csv  model/           sample_submission.csv  test.csv
items.csv            sales_train.csv  shops.csv              tmp/


### Pre-processing

In [5]:
df_train = pd.read_csv(f'{PATH}/sales_train.csv', parse_dates=['date'])

In [6]:
df_train.describe(include='all').T

,count,unique,top,freq,first,last,mean,std,min,25%,50%,75%,max
date,2935849,1034,2013-12-28 00:00:00,9434,2013-01-01 00:00:00,2015-12-10 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN
date_block_num,2.93585e+06,NaN,NaN,NaN,NaN,NaN,14.5699,9.42299,0,7,14,23,33
shop_id,2.93585e+06,NaN,NaN,NaN,NaN,NaN,33.0017,16.227,0,22,31,47,59
item_id,2.93585e+06,NaN,NaN,NaN,NaN,NaN,10197.2,6324.3,0,4476,9343,15684,22169
item_price,2.93585e+06,NaN,NaN,NaN,NaN,NaN,890.853,1729.8,-1,249,399,999,307980
item_cnt_day,2.93585e+06,NaN,NaN,NaN,NaN,NaN,1.24264,2.61883,-22,1,1,1,2169


In [9]:
df_test = pd.read_csv(f'{PATH}/test.csv')
df_shops = pd.read_csv(f'{PATH}/shops.csv')
df_categories = pd.read_csv(f'{PATH}/item_categories.csv')
df_items = pd.read_csv(f'{PATH}/items.csv')
df_submission = pd.read_csv(f'{PATH}/sample_submission.csv')

In [8]:
df_test.tail().T

,214195,214196,214197,214198,214199
ID,214195,214196,214197,214198,214199
shop_id,45,45,45,45,45
item_id,18454,16188,15757,19648,969


In [10]:
df_shops.tail().T

,55,56,57,58,59
shop_name,Цифровой склад 1С-Онлайн,"Чехов ТРЦ ""Карнавал""","Якутск Орджоникидзе, 56","Якутск ТЦ ""Центральный""","Ярославль ТЦ ""Альтаир"""
shop_id,55,56,57,58,59


In [11]:
df_categories.tail().T

,79,80,81,82,83
item_category_name,Служебные,Служебные - Билеты,Чистые носители (шпиль),Чистые носители (штучные),Элементы питания
item_category_id,79,80,81,82,83


In [12]:
df_items.tail().T

,22165,22166,22167,22168,22169
item_name,"Ядерный титбит 2 [PC, Цифровая версия]",Язык запросов 1С:Предприятия [Цифровая версия],Язык запросов 1С:Предприятия 8 (+CD). Хрустале...,Яйцо для Little Inu,Яйцо дракона (Игра престолов)
item_id,22165,22166,22167,22168,22169
item_category_id,31,54,49,62,69


In [13]:
df_submission.tail().T

,214195,214196,214197,214198,214199
ID,214195.0,214196.0,214197.0,214198.0,214199.0
item_cnt_month,0.5,0.5,0.5,0.5,0.5


In [19]:
df_train = df_train.merge(df_items, on='item_id').drop('item_name', axis=1)

In [20]:
add_datepart(df_train, 'date', drop=False)

In [24]:
df_train.columns

Index(['date', 'date_block_num', 'shop_id', 'item_id', 'item_price',
       'item_cnt_day', 'item_category_id', 'Year', 'Month', 'Week', 'Day',
       'Dayofweek', 'Dayofyear', 'Is_month_end', 'Is_month_start',
       'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start',
       'Elapsed'],
      dtype='object')

In [26]:
df_train.groupby(by=['Year', 'Month']).Day.unique()

Year  Month
2013  1        [23, 20, 26, 27, 14, 4, 15, 24, 28, 29, 2, 30,...
      2        [1, 22, 26, 15, 8, 13, 28, 18, 5, 7, 24, 16, 1...
      3        [1, 21, 13, 31, 7, 8, 26, 17, 22, 29, 24, 25, ...
      4        [29, 15, 25, 16, 26, 28, 21, 2, 8, 10, 13, 1, ...
      5        [1, 2, 3, 10, 24, 14, 4, 25, 18, 15, 26, 19, 1...
      6        [1, 23, 25, 12, 27, 7, 2, 21, 20, 29, 15, 10, ...
      7        [14, 21, 20, 4, 31, 9, 12, 2, 3, 5, 29, 28, 16...
      8        [1, 8, 20, 3, 24, 30, 31, 17, 6, 16, 15, 4, 5,...
      9        [13, 16, 22, 5, 8, 27, 24, 28, 15, 1, 2, 26, 1...
      10       [1, 2, 27, 19, 26, 4, 6, 30, 3, 7, 11, 5, 8, 9...
      11       [10, 3, 4, 17, 23, 29, 16, 25, 8, 1, 5, 28, 2,...
      12       [2, 31, 9, 30, 29, 28, 11, 23, 20, 5, 26, 4, 6...
2014  1        [19, 27, 4, 17, 23, 1, 25, 28, 18, 14, 26, 24,...
      2        [22, 7, 19, 1, 15, 18, 28, 17, 14, 20, 27, 13,...
      3        [16, 1, 2, 21, 23, 24, 26, 20, 31, 27, 14, 17,...
      4      

delete the missing months: 2015-11 & 2015-12

In [27]:
df_train.to_feather(f'{TMP_DIR}/train')

In [35]:
len(df_train) - len(df_train[df_train.date < '2015-11-1'].sort_index(by='date', ascending=False))

/home/alex/anaconda3/envs/fastai-cpu/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: by argument to sort_index is deprecated, please use .sort_values(by=...)
  """Entry point for launching an IPython kernel.


39067

2935849